In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
from urllib.request import urlopen

In [3]:
html = urlopen('http://www.100ppi.com/monitor/')

In [4]:
soup = BeautifulSoup(html)

/home/wangshi/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [5]:
d0 = []
for i in soup.find('table',{"bgcolor":"#c7d8ff"}).findAll('td', {'align':'center','bgcolor':'#fafdff'}):
    d0.append(i.get_text().strip('\n'))

In [6]:
d0

['商品名称', '参考规格', '最新价格(2016-07-02)', '上周初价格(2016-06-20)', '上月初价格(2016-06-01)']

In [7]:
d1 = []
for i in soup.find('table',{"bgcolor":"#c7d8ff"}).findAll('td', {'class':'w1'}):
    d1.append(i.parent.get_text().strip('\n').split('\n'))

In [8]:
d2 = pd.DataFrame(d1, columns=d0).dropna()

In [9]:
d2[d0[2]] = d2[d0[2]].apply(pd.to_numeric)
d2[d0[3]] = d2[d0[3]].apply(pd.to_numeric)
d2[d0[4]] = d2[d0[4]].apply(pd.to_numeric)

In [10]:
d2['较上周初变化'] = round((d2[d0[2]]/d2[d0[3]] -1 ) * 100, 2)
d2['较上月初变化'] = round((d2[d0[2]]/d2[d0[4]] -1 ) * 100, 2)
d0.append('较上周初变化')
d0.append('较上月初变化')

In [11]:
d3 = d2[np.abs(d2['较上周初变化']) > 5]

In [28]:
d3.sort(d0[5])

/home/wangshi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,商品名称,参考规格,最新价格(2016-07-02),上周初价格(2016-06-20),上月初价格(2016-06-01),较上周初变化,较上月初变化
130,液氯,品级:合格品 ;氯含量/%(v/v) ≥: 99.6;水分含量/%(m/m) ≤:0.04 ;,220.83,475.00,550.00,-53.51,-59.85
195,钨精矿,品级:一级 ;Wo3含量:≥65% ;类型:黑钨精矿 ;,62250.00,69125.00,73250.00,-9.95,-15.02
186,钼精矿,牌号:kMo-45 ;,916.67,980.00,980.00,-6.46,-6.46
202,仲钨酸铵,Wo3含量:≥88.5% ;牌号:APT-0 ;,100000.00,106666.67,115166.67,-6.25,-13.17
29,丙酮,产品等级:优级品 ;纯度:99.5% ;执行质量标准:国标 ;,4533.33,4833.33,5100.00,-6.21,-11.11
157,氨纶,细度:40D ;光泽:有光 ;,29550.00,31400.00,32050.00,-5.89,-7.80
199,氧化镝,牌号:101020 ;Dy2O3/REO:≥99% ;REO:≥99% ;,1225000.00,1295000.00,1295000.00,-5.41,-5.41
214,钨铁,材质:FeW70 ;规格:70 ;,111500.00,117500.00,119500.00,-5.11,-6.69
251,鸡蛋,颜色:褐壳 ;等级:XL ;包装:箱装 ;,6.10,5.80,6.96,5.17,-12.36
155,顺丁橡胶,牌号:BR9000 ;,10220.00,9686.67,10486.67,5.51,-2.54


In [56]:
d4 = ''
for i in d3.index:
    d4 = d4 + d3[d0[0]][i] + ' ' * abs(4 - len(d3[d0[0]][i])) + '\t'  + str(d3[d0[2]][i]) +'，\t' + '较上周初\t' + str(d3[d0[5]][i]) + '%\t' + ',' + '较上月初\t' + str(d3[d0[6]][i]) + '%' +'\n'
d4 = d4 + 'from ' + '生意社商品监测 ' + 'http://www.100ppi.com/monitor/'

In [57]:
import smtplib
from email.mime.text import MIMEText
from email.header import Header
subject = '生意社商品信息'
smtpserver = 'smtp.139.com'
username = '13730470137'
password = '11223344'
msg = MIMEText(d4,'plain','utf-8')
msg['Subject'] = Header(subject, 'utf-8')
smtp = smtplib.SMTP()
smtp.connect('smtp.139.com')
smtp.login(username, password)

sender = '13730470137@139.com'
receiver = '13731239008@139.com'

smtp.sendmail(sender, receiver, msg.as_string())
smtp.quit()


(221, b'localhost richmail system closing transmission channel')